In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from time import sleep
from datetime import date
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append("../")
import src.herramientas_scrapeo as hs
import src.biblioteca as bb
import src.soporte as sp

In [ ]:
# Configuring options of Selenium
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation']) # To hide as bot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') # Starting with the windows maximized
opciones.add_argument('user.data-dir=selenium') # Saving cookies
opciones.add_argument('--incognito') # incognit

In [2]:
# Declaring some variables for later usage
productos_mercadona = bb.productos_mercadona_estructura.copy() # Structure of the future dataframe
recorrer = bb.listacategorias_merc.copy() # a list of categories that I want to scrap
errores = set() # A set where i store the categories that coundn't been scraped well
today = date.today() # Making a variable with actual Date

On the cell below I make some ajust to scrap better the data that i could scrap, this happens because Mercadona has a anti-bot system.... 😥😥

In [3]:

dfincompleto = pd.read_csv("../data/merc2023-01-20_suma.csv") # Loading the data that I already scraped
for cat_id in dfincompleto["category_id"].unique(): # Searching the categories that was already scraped
    if cat_id in recorrer:
        recorrer.remove(cat_id) # And droping them from the list of categories that i want to scrap
    else:
        pass

# Scraping with a function

In [ ]:
hs.scrap_mercadona(recorrer, productos_mercadona, errores)

# Making it a dataframe

In [ ]:
# With our data with structure of a diccionary I make a dataframe
dfm = pd.DataFrame(productos_mercadona)


In [ ]:
# Making a copy
dfm2 = dfm.copy()

In [ ]:
dfm2.head(2)

In [ ]:
# Checking the number of items that I've scraped
dfmcate_id = dfm2["category_id"].value_counts().reset_index()

In [ ]:
# There is some that didn't scrap well so i search them
dfmcate_id.apply(lambda x: sp.revisar_errores(x["index"], x["category_id"], errores), axis = 1)

In [ ]:
# And after drop them  to scrap then properly later
for error in errores:
    dfm2 = dfm2[dfm2["category_id"] != error]

In [ ]:
# Droping some products that doesn't have all the info that I'm searching for
dfm2.dropna(inplace=True)

# Storing our data

If it's the second try scraping, first, we need to load our CSV where had storaged our data and merge it with the data that we just scraped

In [ ]:

dfsuma = pd.read_csv("../data/merc2023-01-20_suma.csv")
dfsuma2 = pd.concat([dfsuma, dfm2], axis = 0, join = "outer", ignore_index = True)
dfsuma2.to_csv(f"../data/merc{today}_suma.csv")

If it's the first time scraping, we storage our data into a CSV

In [ ]:
dfm2.to_csv(f"../data/merc{today}_suma.csv")